In [70]:
from pyspark.sql import SparkSession

# Start SPARK Session
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [71]:
# import pandas as pd
# import datetime

# # import data
# data = pd.read_csv("/home/bluepi/Downloads/Update/product_info/main table/product_info.csv")

# # convert day column type to datetime
# data['day'] = pd.to_datetime( data['day'], infer_datetime_format=True, yearfirst=True)

# # create new column
# data['date_timestamp'] = pd.to_datetime(data.day.astype(str) + ' ' + data.time)

# # Drop old columns
# data.drop(['day','time'],inplace=True,axis=1)

# # Write to csv
# data.to_csv("/home/bluepi/Downloads/Update/product_info/main table/main table.csv", index=False)

In [81]:
mainTable = spark.read.format('csv').options(
    header=True, inferschema=True).load(
        "/home/bluepi/Downloads/Update/product_info/main table/")

In [83]:
mainTable.printSchema()

root
 |-- p_id: string (nullable = true)
 |-- p_name: string (nullable = true)
 |-- price: string (nullable = true)
 |-- date_timestamp: string (nullable = true)



In [86]:
# Manually set DATA FRAME Schema
from pyspark.sql.types import StructField, IntegerType, StringType, StructType

# Create the list of structure fields--> three parameters ---> Name, DataType, Nulables
data_schema = [
    StructField('p_id', IntegerType(), True),
    StructField('p_name', StringType(), True),
    StructField('price', IntegerType(), True)
]